In [2]:
import pandas as pd
import geopandas
import requests
from requests.structures import CaseInsensitiveDict
import json
import re
import pickle
import os.path
from myConfig import KEY

## Funções Auxiliares

In [3]:
def transform_into_list(data):
    result = list()
    for item in data:
        result.append(item)
    return result

In [4]:
def split_100_elements(x: list):
    """Transforma uma grande lista, em uma lista de listas de 100 elementos em cada posição da respectiva lista.
    """
    quantidade_requisicoes = len(x)//100
    lista_separada = []

    for i in range(0, quantidade_requisicoes+1):
        
        if i == quantidade_requisicoes:
            inicio = i*100
            fim = len(x)
        else:
            inicio = i*100
            fim = ((i+1)*100)

        lista_separada.append(x[inicio:fim])

    return lista_separada

In [5]:
def remove_spaces(x):
    x = str(x)
    return re.sub(r"\s+", " ", x)

In [6]:
def get_LatLong(enderecos: list):
    
    url = f'http://www.mapquestapi.com/geocoding/v1/batch?key={KEY}'

    locations = transform_into_list(enderecos)
    locations = list(map(remove_spaces, locations))
    
    data = {
        "locations": locations,
        "options": {
            "maxResults": -1,
            "thumbMaps": True,
            "ignoreLatLngInput": False
        }
    }

    headers = CaseInsensitiveDict()    

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.request("POST", url, headers=headers, data=json.dumps(data))
    print(response, end='') 
    return response


In [7]:
def extract_LatLng(data):
    final_resposta = []
    for resposta in data:
        resposta = resposta.json()

        for element in resposta['results']:
            temp = [
                element['providedLocation']['street'],
                element['locations'][0]['latLng']['lat'],
                element['locations'][0]['latLng']['lng']
            ]
            final_resposta.append(temp)
            
    return final_resposta

In [8]:
def file_exists(filepatch:str):
    return os.path.exists(filepatch)

## Carrega as tabelas

In [9]:
depencia = ['estaduais', 'federais', 'municipais', 'privadas']
tabelas = {}
for nome in depencia:
    tabelas['df_', nome] = pd.read_excel(f'raw_xlsx/{nome}.xlsx', header=4)

df = pd.concat(tabelas, ignore_index=True)

#### Limpa dados, converte organiza e cria a coluna endereço completo

In [10]:
# Remover colunas, e renomear
unnamed_colluns = ['Unnamed: 11',
                'Unnamed: 12',
                'Unnamed: 13',
                'Unnamed: 14',
                'Unnamed: 15']

rename_colluns = {
    'Nº': 'num',
    'município': 'municipio',
    'Endereço': 'endereco'}
    
df = df.drop(unnamed_colluns, axis=1)
df = df. rename(rename_colluns, axis=1)
df['num'] = df['num'].astype('str')
df['CEP'] = df['CEP'].astype('str') 
df['municipio'] = df['municipio'].str.strip()

df['endereco_completo'] = df['endereco'] + ', ' \
                        + df['num'] + ' - ' \
                        + df['bairro'] + ', ' \
                        + df['municipio'] + \
                        ' - SC, '+ df['CEP']


In [11]:
centenas = split_100_elements(df['endereco_completo'])

#### Processo de busca de coordenadas

In [12]:
data = list()

if not file_exists('data.pkl'):
    for centena in centenas:
        data.append(get_LatLong(centena))
    
    data = extract_LatLng(data)

else:
    print('carregando dados já processados.')
    with open('data.pkl', 'rb') as file:
        data = pickle.load(file)
    print('Dados carregados!')

carregando dados já processados.
Dados carregados!


In [13]:
lat, lng = [], []
for dado in data:
    lat.append(dado[1])
    lng.append(dado[2])

df['lat'] = lat
df['lng'] =lng

In [16]:
df.to_csv('dataframe.csv', sep=';', index=False)